## 정렬, Aggreagation 함수, GroupBy 적용 

DataFrame과 Series의 정렬을 위해서는 sort_values( ) 메서드를 이용한다. sort_values()는 RDBMS SQL의 order by키워드와 유사하다. sort_values()의 주요 입력 파라미터는 by, ascending, inplace이다. by로 특정 칼럼을 입력하면 해당 칼럼으로 정렬을 수행한다. ascending=True로 설정하면 오름차순으로 정렬하며, ascending=False로 설정하면 내림차순으로 설정된다. 기본(default)는 ascendling=True이다. inplace도 default는 inplace=False이다.

In [1]:
import numpy as np
import pandas as pd

In [7]:
titan_df = pd.read_csv('./train.csv')

## Series의 경우에는 Value들이 값으로 호출되어
## sort_values를 그냥 사용하여도 됨
titan_df['Name'].sort_values()

845                      Abbing, Mr. Anthony
746              Abbott, Mr. Rossmore Edward
279         Abbott, Mrs. Stanton (Rosa Hunt)
308                      Abelson, Mr. Samuel
874    Abelson, Mrs. Samuel (Hannah Wizosky)
                       ...                  
286                  de Mulder, Mr. Theodore
282                de Pelsmaeker, Mr. Alfons
361                del Carlo, Mr. Sebastiano
153          van Billiard, Mr. Austin Blyler
868              van Melkebeke, Mr. Philemon
Name: Name, Length: 891, dtype: object

In [13]:
### Dataframe을 정렬할 경우 
titan_df.sort_values(by = 'Age') # 기준이 될 하나의 컬럼을 잡아서 정렬.

## 내림차순 정렬 
# Ascending parameter를 조절
titan_df.sort_values(by = 'Age', ascending=False)

## 1차 오름차순, 2차 오름차순???
titan_df.sort_values(by = ['Name', 'Age'], ascending=[True, False]).head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
845,846,0,3,"Abbing, Mr. Anthony",male,42.0,0,0,C.A. 5547,7.55,NaN,S
746,747,0,3,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.25,NaN,S


### Aggregation 함수적용 

Aggregation :: min(), max(), sum(), count()와 같이 총합 또는 총계처리를 위한 함수 

In [14]:
titan_df.count()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [19]:
### 특정 컬럼들의 평균 및 aggregation을 보고 싶다.
titan_df[['Age','Fare']].mean()

Age     29.699118
Fare    32.204208
dtype: float64

### groupby()적용

In [30]:
titan_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [33]:
## 결측값이 존재하는
# 'Age'와 'Cabin'의 컬럼만 호출 -1
titan_df.groupby(by='Pclass').count()[['Age','Cabin']]

,Age,Cabin
Pclass,,
1,186,176
2,173,16
3,355,12


In [34]:
# 'Age'와 'Cabin'의 컬럼만 호출 -2
titan_df.groupby(by='Pclass')[['Age','Cabin']].count()

,Age,Cabin
Pclass,,
1,186,176
2,173,16
3,355,12


In [37]:
# 'Pclass'에 따른 'Age', 'Fare'의 최소, 최대값을
# groupby를 통해 구해보자.

titan_df.groupby(by='Pclass')[['Age','Fare']].agg(['max','min'])

Age            Fare     
         max   min       max  min
Pclass                           
1       80.0  0.92  512.3292  0.0
2       70.0  0.67   73.5000  0.0
3       74.0  0.42   69.5500  0.0

서로 다른 aggregation 함수를 groupby에서 호출하려면 
SQL은 Select max(Age), sum(SibSp), avg(Fare) from titanic_table group by Pclass와 같이 쉽게 가능하다.

In [38]:
agg_format = {'Age':'max',
              'SibSp':'sum',
             'Fare':'mean'}

titan_df.groupby(by='Pclass').agg(agg_format)

,Age,SibSp,Fare
Pclass,,,
1,80.0,90,84.154687
2,70.0,74,20.662183
3,74.0,302,13.675550


### fillna()로 결측값 데이터 대체

In [42]:
### 결측값의 개수 확인
titan_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [49]:
titan_df['Cabin'] = titan_df['Cabin'].fillna('N') # reassign(재할당)
titan_df['Age'] = titan_df['Age'].fillna(np.mean(titan_df['Age']))
titan_df['Embarked'] = titan_df.Embarked.fillna('N')

titan_df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### apply lambda 식으로 데이터 가공

In [52]:
### 사용자 정의 함수
def get_square(x):
    result = x**2
    return result

In [54]:
print('3의 제곱', get_square(3))

3의 제곱 9


In [57]:
lambda_square = lambda x: x**2
print('lambda로 만든 3의 제곱', lambda_square(3))

lambda로 만든 3의 제곱 9


In [59]:
### map 함수는 참고로 넣어드렸습니다 ^^ 

a= [1,2,3]

result = map(lambda x : x**2, a)
list(result)

[1, 4, 9]

In [74]:
### titan_df의 Name컬럼을 통해
### 각 관측치(obs :: row)의 이름의 길이를 구해봅시다 ^^
###
titan_df['Name'].apply(lambda x : len(x))

titan_df['Na_len'] = titan_df['Name'].str[:].apply(lambda x : len(x))
titan_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Na_len
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,N,S,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,N,S,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,N,S,24


In [84]:
### apply lambda를 모른다면?
## for문으로 풀이가 가능은 하다.

result = []
for i in range(len(titan_df['Name'])):
    result.append(len(titan_df.loc[i,'Name']))
    
titan_df['test'] = result # test명의 컬럼을 생성
titan_df.head()   

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Na_len,test
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,N,S,23,23
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,51,51
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,N,S,22,22
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,44,44
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,N,S,24,24


In [95]:
### Age 컬럼으로 young과 adult를 구분해보자 ^^
titan_df['Age_cat']= titan_df['Age'].apply(lambda x: 'young' if x< 45 else 'adult')
titan_df.Age_cat.value_counts()

young    776
adult    115
Name: Age_cat, dtype: int64

In [96]:
### Age 컬럼으로 young과 adult를 구분해보자 ^^
titan_df['Age_cat']= titan_df['Age'].apply(lambda x: 'young' if x< 30 else ('adult' if x<60 else 'elderly'))
titan_df.Age_cat.value_counts()

young      561
adult      304
elderly     26
Name: Age_cat, dtype: int64

In [107]:
### age_cat을 재정의하는 함수를 만들어보자.
def get_cat(x):
        
    char = ''
    if x <= 5: char = 'baby'
    elif x<= 12: char = 'child'
    elif x<= 18: char ='teen'
    elif x<= 25: char ='student'
    elif x<= 45: char ='young_adult'
    elif x<= 60: char ='adult'
    else: char = 'elderly'       
        
    return char

In [109]:
titan_df['Age_cat'] = titan_df.Age.apply(lambda x: get_cat(x))
titan_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Na_len,test,Age_cat
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,N,S,23,23,student
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,51,51,young_adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,N,S,22,22,young_adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,44,44,young_adult
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,N,S,24,24,young_adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,N,S,21,21,young_adult
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,28,28,student
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,N,S,40,40,young_adult
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,21,21,young_adult


In [110]:
# end of file